# Crear y administrar una base de datos no relacional

### Información general
En este tutorial, usted crea una tabla DynamoDB y utiliza la tabla para almacenar y recuperar datos. Utiliza Python y Boto 3, el AWS SDK para Python, para interactuar con las API de DynamoDB. También aprende algunas maneras claves en las que DynamoDB (una base de datos no relacional) difiere de las bases de datos relacionales tradicionales. El tutorial utiliza una aplicación de librería en línea como un ejemplo guía.

### Por qué es importante
Deberá entender cómo modelar sus datos para que se ajusten tanto a los patrones de acceso a datos de su aplicación como al modelo subyacente de su base de datos. Al diseñar una nueva aplicación, desea saber que su elección de base de datos puede manejar la mayor carga proveniente del uso adicional de su aplicación.

DynamoDB proporciona un rendimiento de baja latencia con escalamiento casi infinito, por lo que no necesita preocuparse por los obstáculos de rendimiento a medida que su aplicación crece. Se puede acceder a DynamoDB a través de una API HTTP o un punto de enlace HTTPS, proveyendo un modelo de interacción simple y seguro con su base de datos. Finalmente, DynamoDB es una base de datos NoSQL, que permite un diseño de esquema flexible que puede evolucionar con la aplicación.

In [42]:
import sys
sys.path.append('../')
from decimal import Decimal
from io import BytesIO
from datetime import datetime
import ast
import json
import logging
import os
from pprint import pprint
import requests
from zipfile import ZipFile
import boto3
import pandas as pd
from dynamodb import DynamoTable

In [64]:
from importlib import reload
import dynamodb
reload(dynamodb)
from dynamodb import DynamoTable

### Información general

Suponga que va a crear una aplicación de una librería en línea. La librería tiene un gran inventario de libros que se almacenan en su almacén para ser vendidos a los clientes. La aplicación debe mostrar a los usuarios los libros que están disponibles para la venta, así como información básica acerca del libro, como el autor.

La aplicación debe recuperar un libro específico en función del título y el autor a fin de que el usuario pueda explorar más detalles cuando vea el libro. Además, la aplicación debe permitir a los usuarios navegar por todos los libros de una categoría específica, como Historia o biografías, para permitir el descubrimiento de libros interesantes.

Finalmente, puede tener diferentes formatos de un libro, como en tapa dura, tapa blanda o audiolibro. La aplicación debe almacenar los formatos de libros en existencia mediante un mapeo asociado a los números de elemento del sistema de inventario. La aplicación debe permitir la actualización de los formatos a través del tiempo, puesto que es posible que se necesiten agregar o eliminar formatos de un libro determinado.

### Terminología
Los siguientes conceptos de DynamoDB son importantes para este módulo:

* **Tabla**: un conjunto de registros de datos de DynamoDB.

* **Elemento**: un solo registro de datos en una tabla de DynamoDB. Es similar a una fila en una base de datos relacional.

* **Atributo**: un único componente de datos que corresponde a un elemento. Es similar a una columna en una base de datos relacional. Sin embargo, a diferencia de las columnas de las bases de datos relacionales, no es necesario especificar los atributos al momento de crear la tabla, salvo la clave principal que se aborda más adelante en este módulo. Los atributos pueden ser de tipo simple, como cadenas, valores enteros o booleanos, así como de tipo complejo, tales como listas o mapas.

* **Clave principal**: una clave principal es un identificador único de un solo elemento en una tabla de DynamoDB. Tanto el nombre de la clave principal como el tipo se deben especificar al momento de crear la tabla. Además, se debe incluir una clave principal del tipo especificado con cada elemento incluido en la tabla. Una clave principal simple está compuesta por un único atributo, mientras que una clave principal compuesta cuenta con dos atributos: una clave de partición y una clave de ordenación. Por ejemplo, puede crear una clave principal simple con “UserID” como identificador o crear una clave principal compuesta con la combinación de “UserID” y “Creation_Date” como identificador de un elemento.

### Modelo de datos

Al momento de crear una aplicación, siempre debe dedicar tiempo al diseño de los modelos de datos necesarios para la lógica de la aplicación. El diseño del modelo de datos debe tener en cuenta las necesidades de acceso a los datos que la aplicación requerirá, tanto para leer como para escribir datos.

DynamoDB es una base de datos no relacional. Si se utilizan bases de datos no relacionales, no es necesario especificar el esquema completo de forma anticipada al crear la tabla. Solo es necesario presentar la clave principal para la tabla, la cual identifica cada registro de la tabla de forma exclusiva. Esto reduce el monto de los costos iniciales correspondientes al diseño del modelo de datos, ya que es posible modificar el esquema fácilmente a medida que cambian las necesidades de la aplicación.

Como se mencionó en la sección “Contexto de la aplicación” de la “Introducción” a este tutorial, la aplicación debe recuperar un libro específico según el título y el autor. Debido a que la combinación del título y el autor representan el identificador único de un libro, puede utilizar esos atributos como la clave principal de la tabla. La aplicación también debe almacenar información sobre la categoría del libro, como historia o biografía, así como información acerca de los formatos disponibles (encuadernación de tapa dura y tapa blanda, o audiolibro). Esa información se asigna a los números del elemento en el sistema que funciona como inventario.

En este contexto, puede utilizar el siguiente esquema para la tabla:

* Título (una cadena): el título del libro
* Autor (una cadena): el autor del libro
* Categoría (una cadena): la categoría a la que pertenece el libro, por ejemplo, historia, biografía y ciencia ficción
* Formatos (un mapa): los diferentes formatos que están disponibles para la venta (por ejemplo, encuadernación de tapa dura y tapa blanda, o audiolibro) y los números del elemento correspondientes en el sistema que funciona como inventario

In [65]:
dt = DynamoTable()

In [67]:
dt.select_table('Books')

In [66]:
dt.create_table(
    table_name="Books",
    partition_key="Author",
    partition_key_type="S",
    sort_key="Title",
    sort_key_type="S"
)  

Couldn't create table Books. Here's why: ResourceInUseException: Table already exists: Books


In [46]:
df = pd.read_csv('books_gr.csv', on_bad_lines='skip')

data_col = list(df.columns)
ref_col = []
for i in data_col:
    if i == "authors":
        i = "Author"
    ref_col.append(i.replace(" ", "").capitalize())
df.columns = ref_col
df.drop_duplicates(subset=["Title", "Author"], inplace=True)

df.head()

,Bookid,Title,Author,Average_rating,Isbn,Isbn13,Language_code,Num_pages,Ratings_count,Text_reviews_count,Publication_date,Publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [47]:
dt.batch_pandas(df)

KeyboardInterrupt: 

In [ ]:
dt.get_item(
    pk_value="Charles Simic",
    se_value="Dime-Store Alchemy: The Art of Joseph Cornell"
)

{'Num_pages': 116.0,
 'Isbn13': 9781590171707.0,
 'Publication_date': '9/12/2006',
 'Isbn': '1590171705',
 'Text_reviews_count': 12.0,
 'Language_code': 'eng',
 'Title': 'Dime-Store Alchemy: The Art of Joseph Cornell',
 'Bookid': 45336.0,
 'Ratings_count': 70.0,
 'Author': 'Charles Simic',
 'Average_rating': 4.18,
 'Publisher': 'NYRB Classics'}

Debido a que cada elemento de la tabla se identifica de forma exclusiva con su clave principal, la llamada a la API GetItem siempre devolverá como máximo un elemento de la tabla.

In [ ]:
dt.query_items("Susan Cooper", to_pandas=True)

,Num_pages,Isbn13,Publication_date,Isbn,Text_reviews_count,Language_code,Title,Bookid,Ratings_count,Author,Average_rating,Publisher
0,167,9780020421900,4/30/1987,0020421907,72,eng,Seaward,11308,1756,Susan Cooper,3.97,Aladdin Paperbacks
1,274,9780689840333,12/1/2000,0689840330,507,eng,Silver on the Tree (The Dark is Rising #5),11313,27802,Susan Cooper,4.15,Margaret K. McElderry Books
2,786,9780140316889,10/25/1984,0140316884,16,eng,The Dark Is Rising Sequence,24958,266,Susan Cooper,4.28,Puffin Books (Penguin Books)


Tanto la llamada a la API GetItem para obtener un único libro como la llamada a la API query_items() para recuperar todos los libros escritos por un autor utilizan la clave primaria especificada en la tabla de Libros. Sin embargo, es posible que desee habilitar patrones de acceso adicionales, como recuperar todos los libros de una categoría, como historia o biografías. La Categoría no hace parte de la clave primaria de la tabla, pero se puede crear un índice secundario a fin de permitir patrones de acceso adicionales.


In [ ]:
dt.query_items(
    query="Chaim Potok", 
    to_pandas=True, 
    consumed_capacity=True
)

Consumed Capacity: 0.5


,Num_pages,Isbn13,Publication_date,Isbn,Text_reviews_count,Language_code,Title,Bookid,Ratings_count,Author,Average_rating,Publisher
0,371,9780449911839,8/27/1996,0449911837,314,eng,Davita's Harp,14769,5005,Chaim Potok,3.99,Ballantine Books
1,416,9780449001134,9/10/1997,044900113X,120,eng,In the Beginning,11503,2222,Chaim Potok,4.08,Ballantine Books
2,369,9781400031047,3/11/2003,1400031044,2398,eng,My Name Is Asher Lev,11507,32785,Chaim Potok,4.21,Anchor
3,384,9780449001158,9/10/1997,0449001156,339,eng,The Gift of Asher Lev,11502,6224,Chaim Potok,4.16,Ballantine Books
4,368,9781400095414,11/8/2005,1400095417,499,eng,The Promise,11499,11255,Chaim Potok,4.15,Anchor Books


### Crear un índice secundario

DynamoDB permite la creación de índices secundarios para obtener más patrones de acceso a los datos en la tabla. Los índices secundarios son un medio eficaz para agregar flexibilidad de consulta a la tabla de DynamoDB.

DynamoDB tiene dos tipos de índices secundarios: los índices secundarios globales y los índices secundarios locales. En esta sección, se agrega un índice secundario global al atributo de categoría. Este permitirá recuperar todos los libros de una categoría específica.

In [ ]:
client = boto3.client('dynamodb')

In [ ]:
try:
    resp = client.update_table(
        TableName="Books",
        # Any attributes used in your new global secondary index must be declared in AttributeDefinitions
        AttributeDefinitions=[
            {
                "AttributeName": "Publisher",
                "AttributeType": "S"
            },
        ],
        # This is where you add, update, or delete any global secondary indexes on your table.
        GlobalSecondaryIndexUpdates=[
            {
                "Create": {
                    # You need to name your index and specifically refer to it when using it for queries.
                    "IndexName": "PublisherIndex",
                    # Like the table itself, you need to specify the key schema for an index.
                    # For a global secondary index, you can use a simple or composite key schema.
                    "KeySchema": [
                        {
                            "AttributeName": "Publisher",
                            "KeyType": "HASH"
                        }
                    ],
                    # You can choose to copy only specific attributes from the original item into the index.
                    # You might want to copy only a few attributes to save space.
                    "Projection": {
                        "ProjectionType": "ALL"
                    },
                    # Global secondary indexes have read and write capacity separate from the underlying table.
                    # If not selected PAY_PER_REQUEST when create table
                    # "ProvisionedThroughput": {
                    #     "ReadCapacityUnits": 5,
                    #     "WriteCapacityUnits": 5,
                    # }
                }
            }
        ],
    )
    print("Secondary index added!")
except Exception as e:
    print("Error updating table:")
    print(e)

Secondary index added!


Crear un índice secundario global es muy parecido a crear una tabla. Se especifican el nombre del índice, los atributos que estarán dentro del índice, el esquema de la clave del índice y el rendimiento aprovisionado (la capacidad máxima que una aplicación puede consumir de una tabla o de un índice). El rendimiento aprovisionado en cada índice es independiente del rendimiento aprovisionado en una tabla. Esto permite definir el rendimiento con detalle a fin de satisfacer las necesidades de la aplicación.

In [53]:
import time
from boto3.dynamodb.conditions import Key

# Boto3 is the AWS SDK library for Python.
# The "resources" interface allows for a higher-level abstraction than the low-level client interface.
# For more details, go to http://boto3.readthedocs.io/en/latest/guide/resources.html
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('Books')

# When adding a global secondary index to an existing table, you cannot query the index until it has been backfilled.
# This portion of the script waits until the index is in the “ACTIVE” status, indicating it is ready to be queried.
while True:
    if not table.global_secondary_indexes or table.global_secondary_indexes[0]['IndexStatus'] != 'ACTIVE':
        print('Waiting for index to backfill...')
        time.sleep(10)
        table.reload()
    else:
        break

# When making a Query call, you use the KeyConditionExpression parameter to specify the hash key on which you want to query.
# If you want to use a specific index, you also need to pass the IndexName in our API call.
resp = table.query(
    # Add the name of the index you want to use in your query.
    IndexName="PublisherIndex",
    KeyConditionExpression=Key('Publisher').eq('SaltRiver'),
)

print("The query returned the following items:")
for item in resp['Items']:
    print(item)

The query returned the following items:
{'Num_pages': Decimal('234'), 'Isbn13': Decimal('9781414306346'), 'Publication_date': '2/17/2006', 'Isbn': '1414306342', 'Text_reviews_count': Decimal('94'), 'Language_code': 'eng', 'Title': 'Looking for God in Harry Potter', 'Bookid': Decimal('15190'), 'Ratings_count': Decimal('1347'), 'Author': 'John Granger', 'Average_rating': Decimal('4.13'), 'Publisher': 'SaltRiver'}


In [68]:
dt

- Table name: Books            
- Table arn: arn:aws:dynamodb:us-east-1:830206065100:table/Books            
- Table creation: 2022-12-16 16:44:15.221000-03:00            
- Billing mode: PAY_PER_REQUEST            
- [{'AttributeName': 'Author', 'KeyType': 'HASH'}, {'AttributeName': 'Title', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Author', 'AttributeType': 'S'}, {'AttributeName': 'Publisher', 'AttributeType': 'S'}, {'AttributeName': 'Title', 'AttributeType': 'S'}]

### Actualización de elementos

Además de escribir y leer datos, se busca actualizar regularmente los datos existentes en la base de datos. Es preferible contar con la capacidad de actualizar estos datos mediante una sola llamada a la API, en lugar de primero leer un elemento y luego escribir de regreso el elemento completo con las actualizaciones hechas. DynamoDB permite la actualización de los datos in situ mediante la llamada a la API UpdateItem.

Por ejemplo, recuerde que cada elemento tiene un atributo de Formatos. Este es un mapa de todos los diferentes formatos que la librería tiene de un título específico. Con el tiempo, este mapa se debe actualizar para un libro específico, ya sea porque se ha agregado un nuevo formato o porque ya no se utiliza un formato.

In [ ]:
dt.